<a href="https://colab.research.google.com/github/Ayushnema704/flipgrid/blob/main/flipgrid_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
swoyam2609_fresh_and_stale_classification_path = kagglehub.dataset_download('swoyam2609/fresh-and-stale-classification')

print('Data source import complete.')


100%|██████████| 2.84G/2.84G [01:49<00:00, 27.8MB/s]

Extracting files...


Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [1]:
import tensorflow as tf
import os

# Specify the classes to include (excluding the specified fruits/vegetables)
included_classes = [
    'freshbanana',
    'freshcapsicum',
    'freshcucumber',
    'freshoranges',
    'freshpotato',
    'freshtomato',
    'rottenbanana',
    'rottencapsicum',
    'rottencucumber',
    'rottenoranges',
    'rottenpotato',
    'rottentomato'
]

img_height, img_width = 180, 180
batch_size = 64
dataset_dir = "/kaggle/input/fresh-and-stale-classification/dataset/Train"

# Function to count the number of images in each class
def count_images_per_class(directory, classes):
    class_counts = {}
    for class_name in classes:
        class_dir = os.path.join(directory, class_name)
        if os.path.exists(class_dir):
            image_count = len(os.listdir(class_dir))
            class_counts[class_name] = image_count
        else:
            class_counts[class_name] = 0  # If the class directory doesn't exist
    return class_counts

# Count images in each class
image_counts = count_images_per_class(dataset_dir, included_classes)

# Print the counts for each class
print("Number of images per class:")
for class_name, count in image_counts.items():
    print(f"{class_name}: {count} images")

# Load the training dataset while excluding certain classes
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=included_classes  # Include only the specified classes
)

# Load the validation dataset using the same classes
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=included_classes  # Include only the specified classes
)


Number of images per class:
freshbanana: 0 images
freshcapsicum: 0 images
freshcucumber: 0 images
freshoranges: 0 images
freshpotato: 0 images
freshtomato: 0 images
rottenbanana: 0 images
rottencapsicum: 0 images
rottencucumber: 0 images
rottenoranges: 0 images
rottenpotato: 0 images
rottentomato: 0 images


NotFoundError: Could not find directory /kaggle/input/fresh-and-stale-classification/dataset/Train

In [ ]:
from tensorflow.keras import layers, models

# Load the pretrained ResNet50 model without the top layer
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3),
    pooling='avg',
    weights='imagenet'
)

# Freeze the layers in the pretrained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Create a new input layer
inputs = layers.Input(shape=(180, 180, 3))

# Pass the inputs through the pretrained model
x = pretrained_model(inputs)

# Add your custom layers on top
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(12, activation='softmax')(x)

# Create the full model
resnet_model = models.Model(inputs, outputs)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
resnet_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 12)                  │           6,156 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,642,956 (94.01 MB)

 Trainable params: 1,055,244 (4.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Compile the model
resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),  # Optimizer with the initial learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
epochs=10
history = resnet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
    callbacks=EarlyStopping(
    monitor='val_loss',  # You can also monitor 'val_accuracy'
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restores the model weights from the best epoch
)
)

NameError: name 'train_ds' is not defined

In [ ]:
# Save model weights with the required filename ending
model_weights_path = '/kaggle/working/resnet_model_weights.weights.h5'  # Ensure it ends with .weights.h5
resnet_model.save_weights(model_weights_path)
print(f"Model weights saved to {model_weights_path}")


FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = '/kaggle/working/resnet_model_weights.weights.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
import shutil

# Zip the model weights
shutil.make_archive('/kaggle/working/resnet_model_weights', 'zip', '/kaggle/working', 'resnet_model_weights.weights.h5')
print("Model weights zipped and ready for download.")


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/resnet_model_weights.weights.h5'

In [ ]:
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

# Number of images per class
class_counts = {
    'freshbanana': 2468,
    'freshcapsicum': 990,
    'freshcucumber': 496,
    'freshoranges': 1466,
    'freshpotato': 536,
    'freshtomato': 1858,
    'rottenbanana': 2932,
    'rottencapsicum': 901,
    'rottencucumber': 421,
    'rottenoranges': 1595,
    'rottenpotato': 802,
    'rottentomato': 1825
}

# Total number of images
total_images = sum(class_counts.values())
# Calculate class weights
class_weights = {i: total_images / (len(class_counts) * count) for i, count in enumerate(class_counts.values())}

# Load the pretrained ResNet50 model without the top layer
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(180, 180, 3),
    pooling='avg',
    weights='imagenet'
)

# Freeze the layers in the pretrained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Create a new input layer
inputs = layers.Input(shape=(180, 180, 3))

# Pass the inputs through the pretrained model
x = pretrained_model(inputs)

# Add your custom layers on top
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(len(class_counts), activation='softmax')(x)  # Adjusted for number of classes

# Create the full model
resnet_model = models.Model(inputs, outputs)

# Compile the model
resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Define the number of epochs
epochs = 10

# Train the model with class weights
history = resnet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    class_weight=class_weights,  # Add class weights here
    callbacks=[EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )]
)

# Print the model summary
resnet_model.summary()


NameError: name 'train_ds' is not defined

In [ ]:
# Save model weights with the required filename ending
model_weights_path = '/kaggle/working/resnet_model_weights.weights.h5'  # Ensure it ends with .weights.h5
resnet_model.save_weights(model_weights_path)
print(f"Model weights saved to {model_weights_path}")


Model weights saved to /kaggle/working/resnet_model_weights.weights.h5
